## 項目 76 関係する振る舞いを TestCase サブクラスで検証する

Python でテストを書く正しい標準的な方法は、組み込みモジュール unittest を使うことである。

例えば、utils.py で定義された、次のようなユーティリティ関数を、さまざまな入力に対して正しく動作するか検証しようとする。


In [3]:
## utils.py ##
def to_str(data):
    if isinstance(data, str):
        return data
    elif isinstance(data, bytes):
        return data.decode('utf-8')
    else:
        raise TypeError('Must supply str or bytes, '
                        'found: %r' % data)

テストを定義するために、 期待する各振る舞いのテストを含んだ test_utils.py または utils_test.py という名の第2のファイルを作る (ファイル名は任意) 。

In [6]:
## utils_test.py ##
from unittest import TestCase, main
# from utils import to_str

class UtilsTestCase(TestCase):
    def test_to_str_bytes(self):
        self.assertEqual('hello', to_str(b'hello'))

    def test_to_str_str(self):
        self.assertEqual('hello', to_str('hello'))

    def test_failing(self):
        self.assertEqual('incorrect', to_str('hello'))

# if __name__ == '__main__':
#     main()


テストファイルを Python コマンドラインで実行する。この場合、テストメソッドの2つは無事パスするが、1つは失敗し、何がいけなかったのかを示すエラーメッセージが表示される。

```
utils_test.py
F..
======================================================================
FAIL: test_failing (__main__.UtilsTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\utils_test.py", line 28, in test_failing
    self.assertEqual('incorrect', to_str('hello'))
AssertionError: 'incorrect' != 'hello'
- incorrect
+ hello


----------------------------------------------------------------------
Ran 3 tests in 0.001s

FAILED (failures=1)
```

テストは、TestCaseサブクラスで構成する (上の例では UtilsTestCase)。各テストケースは、test_ という語で始まるメソッドである。テストメソッドが何の Exception (assert 文の AssertionError も含めて) も起こさずに実行できた場合、テストは成功したと考えられる。1つのテストが失敗した場合、TestCase サブクラスは最初の問題点で停止するのではなく、継続して他のテストメソッドを実行し、テスト全体の様子がわかるようにする。

もし、特定のテストで迅速にバグを修正したり、改善を反復したい場合には、コマンドラインのテストモジュールで、そのパスを指定して、そのテストメソッドだけを実行することができる。


```
$ python3 utils_test.py UtilsTestCase.test_to_str_bytes
.
---------------------------------------------------------
Ran 1 test in 0.000s

OK
```

テストメソッドの中で、特定のブレークポイントで障害の原因をより深く調べるために直接デバッガを呼び出すこともできる (どのように行うかは「項目 80 pdbで対話的にデバッグすることを考える」参照)。

TestCase クラスは、テストで確認のためのアサーションを行うためのヘルパー関数を提供する。等価性を検証する assertEqual、論理式を検証する assertTrue など多数ある (他は help(TestCase) 参照)。これらは、入出力すべてを出力して、テストが失敗した理由をしっかりと理解できるため、組み込みの assert 文よりも有益である。例えば次に、ヘルパーアサーションメソッドがある場合とない場合とで同じテストケースを比較する。


In [ ]:
from unittest import TestCase, main
from utils import to_str

class AssertTestCase(TestCase):
    def test_assert_helper(self):
        expected = 12
        found = 2 * 5
        self.assertEqual(expected, found)

    def test_assert_statement(self):
        expected = 12
        found = 2 * 5
        assert expected == found

# if __name__ == '__main__':
#     main()

次の失敗メッセージを比較すると分かるよう、ヘルパー関数 assertionEqual を用いたケースの方が原因が判別しやすいだろう。

```
assert_test.py
FF
======================================================================
FAIL: test_assert_helper (__main__.AssertTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\assert_test.py", line 24, in test_assert_helper
    self.assertEqual(expected, found)
AssertionError: 12 != 10
```

```
======================================================================
FAIL: test_assert_statement (__main__.AssertTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\assert_test.py", line 29, in test_assert_statement
    assert expected == found
AssertionError

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=2)
```

with 文でコンテキストマネジャーとして使うことができる例外を検証する assertRaises ヘルパーメソッドもある(どのように動作するかは「項目 66 contextlib と with 文を try/finally の代わりに考える」参照)。これは、try/except 文とよく似ており、どこで例外が発生すると期待されるかを明確にする。


In [7]:
## utils_error_test.py ##
from unittest import TestCase, main

class UtilsErrorTestCase(TestCase):
    def test_to_str_bad(self):
        with self.assertRaises(TypeError):
            to_str(object())

    def test_to_str_bad_encoding(self):
        with self.assertRaises(UnicodeDecodeError):
            to_str(b'\xfa\xfa')

# if __name__ == '__main__':
#     main()


```
python utils_error_test.py
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK
```

テストをより読みやすくするために、複雑なロジックを持つ専用のヘルパーメソッドを TestCase サブクラスで定義できる。ただし、メソッド名が test という語から始まらないようにしないと、テストケースと間違われて実行されてしまうので、その点は要確認である。こういったカスタマイズしたテストヘルパーでは、TestCase アサーションを呼び出すだけでなく、しばしば fail メソッドを使って、どこで仮定や不変条件が成り立たなくなったかを明らかにする。

例えば、ジェネレータの振る舞いを検証するカスタムヘルパーメソッドを次のように定義する。

In [12]:
## helper_test.py ##
from unittest import TestCase, main

def sum_squares(values):
    cumulative = 0
    for value in values:
        cumulative += value ** 2
        yield cumulative

class HelperTestCase(TestCase):
    def verify_complex_case(self, values, expected):
        expect_it = iter(expected)
        found_it = iter(sum_squares(values))
        test_it = zip(expect_it, found_it)

        for i, (expect, found) in enumerate(test_it):
            self.assertEqual(
                expect,
                found,
                f'Index {i} is wrong')

        # Verify both generators are exhausted
        try:
            next(expect_it)
        except StopIteration:
            pass
        else:
            self.fail('Expected longer than found')

        try:
            next(found_it)
        except StopIteration:
            pass
        else:
            self.fail('Found longer than expected')

    def test_wrong_lengths(self):
        values = [1.1, 2.2, 3.3]
        expected = [
            1.1**2,
        ]
        self.verify_complex_case(values, expected)

    def test_wrong_results(self):
        values = [1.1, 2.2, 3.3]
        expected = [
            1.1**2,
            1.1**2 + 2.2**2,
            1.1**2 + 2.2**2 + 3.3**2 + 4.4**2,
        ]
        self.verify_complex_case(values, expected)

# if __name__ == '__main__':
#     main()


ヘルパーメソッドは、テストケースを短く読みやすくし、出力されるエラーメッセージがわかりやすくしている。

```
python helper_test.py
FF
======================================================================
FAIL: test_wrong_lengths (__main__.HelperTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\helper_test.py", line 57, in test_wrong_lengths
    self.verify_complex_case(values, expected)
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\helper_test.py", line 50, in verify_complex_case
    self.fail('Found longer than expected')
AssertionError: Found longer than expected

======================================================================
FAIL: test_wrong_results (__main__.HelperTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\helper_test.py", line 66, in test_wrong_results
    self.verify_complex_case(values, expected)
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\helper_test.py", line 32, in verify_complex_case
    self.assertEqual(
AssertionError: 36.3 != 16.939999999999998 : Index 2 is wrong

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=2)
```

通常、関係するテストセットごとに1つの TestCase サブクラスを定義する。
時には、多くのテストケースを持つ関数ごとに1つの TestCase サブクラスということもある。
他のケースとしては、TestCase サブクラスが単一モジュールのすべての関数を処理することもある。各基底クラスとそのすべてのメソッドをテストする TestCase サブクラスを作ることもしばしばある。

TestCase クラスは、1つのテストメソッドの中で複数のテストを定義するときに、ボイラープレート (使い回す前提で用意されたテキスト) を避けるための subTest ヘルパーメソッドも提供する。これは特に、データ駆動テストを書く場合に役立つ。これは、(多数のテストメソッドを含む TestCase の振る舞いと同様に) ケースのいずれかで失敗しても他のケースのテストを継続するテストメソッドの実行を許可する。

次のデータ駆動テストの例で示す。


In [11]:
## data_driven_test.py ##
from unittest import TestCase, main

class DataDrivenTestCase(TestCase):
    def test_good(self):
        good_cases = [
            (b'my bytes', 'my bytes'),
            ('no error', b'no error'),  # This one will fail
            ('other str', 'other str'),
        ]
        for value, expected in good_cases:
            with self.subTest(value):
                self.assertEqual(expected, to_str(value))

    def test_bad(self):
        bad_cases = [
            (object(), TypeError),
            (b'\xfa\xfa', UnicodeDecodeError),
        ]
        for value, exception in bad_cases:
            with self.subTest(value):
                with self.assertRaises(exception):
                    to_str(value)

# if __name__ == '__main__':
#     main()

no error テストは失敗し、有用なエラーメッセージを出力するが、他のすべてのケースは引き続きテストされ、パスすることが確認された。


```
python data_driven_test.py
.
======================================================================
FAIL: test_good (__main__.DataDrivenTestCase) [no error]
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\data_driven_test.py", line 29, in test_good
    self.assertEqual(expected, to_str(value))
AssertionError: b'no error' != 'no error'

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=1)
```

Note:

プロジェクトの複雑さとテストの要求にもよるが、 pytest (https://pytest.org/) オープン ソースパッケージとその膨大なコミュニティプラグインが特に役立つ。

### 覚えておくこと

- 組み込みモジュール unittest の TestCase クラスのサブクラスでテストを定義でき、テストしたい振る舞いごとにメソッドを定義できる。TestCase クラスにおけるテストメソッドは、test という語から始まらなければならない。
- TestCase クラスで定義された assertEqual のようなさまざまなヘルパーメソッドを使い、組み込みの assert 文を使う代わりに、テストにおける期待する振る舞いを確認できる。
- subTest ヘルパーメソッドを使い、ボイラープレートを減らしてデータ駆動テストを書くことを考える。
